algoritmo que: 
* a gestão de filas de espera em hospitais envolve a classificação de risco e priorização dos pacientes;
* monitorar a quantidade de pessoas inscritas e o tempo de espera;
* definir a prioridade de atendimento. Cada cor tem um tempo de espera tolerável, que indica a gravidade do caso:
    * Vermelho: risco de morte, atendimento imediato;
     * Laranja: tempo de espera tolerável de até 10 minutos;
    * Amarelo: tempo de espera tolerável de até 20 minutos;
    * Verde: não há risco de morte, tempo de espera tolerável de até 60 minutos;
    * Azul: quadros crônicos, encaminhamento ao centro de saúde, tempo de espera tolerável de até 180 minutos.
* organize a agenda de pacientes, devendo considerar:
    * o número de consultas marcadas;
    * o tempo médio de duração e a previsão de contratempos.
* a capacitação dos profissionais de saúde influencia no tamanho das filas.

Em observação à lei abaixo, esta será considerada na implementação do algoritmo:
* A Lei 10.048/00 garante atendimento prioritário a idosos com 60 anos ou mais na administração pública.
* Também inclui pessoas do seguinte perfil: deficientes, gestantes, lactantes, pessoas com crianças de colo e obesos.
* Todavia, indivíduos com idade 80+ tem preferência sob todos os demais.

In [69]:
# !pip install faker
from faker import Faker
import random

import time

import pandas as pd
import datetime

### Gerando dados para teste

In [62]:
# Inicializando o Faker e configurando para nomes em português
fake = Faker("pt_BR")

# Inicializando as listas

lista_nomes = []          # string: primeiro nome
lista_dt_nascimento = []  # date: a data de nascimento
lista_prioritario = []    # bool: False para NÃO; True para SIM
lista_dt_entrada = []     # datetime, indicando o horário de cadastro
lista_especialidade = []  # string: especialidade médica (geral, neurologia, oftalmologista etc)
lista_tma = []            # int: minutos com o tempo médio de atendimento para cada especialidade
lista_class_risco = []    # string: vermelho, laranja, amarelo, verde, azul

lista_colunas = ['Nome', 'Nascimento', 'Prioritário', 'Entrada', 'Especialidade', 'TMA', 'Risco']
df = pd.DataFrame(columns=colunas)

# Opções para os campos
especialidades_exemplo = ["geral", "neurologia", "oftalmologia", "cardiologia", "dermatologia", "pediatria"]
class_risco_exemplo = ["vermelho", "laranja", "amarelo", "verde", "azul"]

# Função para gerar uma data de nascimento aleatória
def gerar_data_nascimento():
    start_date = datetime.date(1940, 1, 1)
    end_date = datetime.date(2005, 12, 31)
    return start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))

# Função para gerar uma data de entrada aleatória com horário
def gerar_data_entrada():
    now = datetime.datetime.now()
    delta_days = random.randint(0, 5)
    delta_seconds = random.randint(0, 86400)  # Número de segundos no dia
    return now - datetime.timedelta(days=delta_days, seconds=delta_seconds)

# Populando as listas com 100 registros
for _ in range(100):
    lista_nomes.append(fake.first_name())  # Gera um primeiro nome aleatório em português
    lista_dt_nascimento.append(gerar_data_nascimento())
    lista_prioritario.append(random.choice([True, False]))
    lista_dt_entrada.append(gerar_data_entrada())
    lista_especialidade.append(random.choice(especialidades_exemplo))
    lista_tma.append(random.choice([15, 30, 45, 60]))  # Tempo médio de atendimento entre 10 e 60 minutos
    lista_class_risco.append(random.choice(class_risco_exemplo))
    
lista_dados = [lista_nomes, lista_dt_nascimento, lista_prioritario, lista_dt_entrada,
               lista_especialidade, lista_tma, lista_class_risco]
    
for i, coluna in enumerate(lista_colunas):
    df[coluna] = lista_dados[i]
    
df

,Nome,Nascimento,Prioritário,Entrada,Especialidade,TMA,Risco
0,Eduarda,1947-02-23,True,2024-10-26 00:08:41.940196,cardiologia,30,verde
1,Carolina,1984-10-29,False,2024-10-23 18:55:00.940196,dermatologia,60,laranja
2,Cauã,1970-01-26,False,2024-10-23 17:55:13.940196,pediatria,60,amarelo
3,Joana,1967-11-11,True,2024-10-24 08:26:08.940196,cardiologia,15,verde
4,Vinícius,1995-11-17,True,2024-10-28 05:22:56.940196,dermatologia,60,verde
...,...,...,...,...,...,...,...
95,Maria Cecília,1949-04-15,False,2024-10-26 13:18:44.943197,neurologia,30,verde
96,Zoe,1982-05-30,False,2024-10-26 02:11:02.943197,dermatologia,60,azul
97,Natália,1987-03-21,True,2024-10-26 21:51:15.943197,neurologia,30,amarelo
98,Thiago,1958-12-10,True,2024-10-27 02:52:53.943197,neurologia,60,vermelho


In [90]:
class Fila():
    def __init__(self):
        self.data = []

    def inserir(self, x):
        self.data.append(x)

    def remover(self):
        if len(self.data) > 0:
            return self.data.pop(0)

    def topo(self):
        if len(self.data) > 0:
            return self.data[0]

    def vazio(self):
        return not len(self.data) > 0

    def exibir(self):
        return list(self.data)
    
    
    
def calc_idade(nascido):
    """
    Calcula a idade de um cliente com base na data de nascimento fornecida.

    A função calcula a idade atual do cliente subtraindo o ano de nascimento do
    ano atual. Ajusta a idade caso o mês e o dia de nascimento ainda não tenham
    ocorrido no ano atual.

    Parâmetros:
    - nascido (datetime.date): A data de nascimento do cliente.

    Retorna:
    - int: A idade do usuário em anos.
    """
    
    now = datetime.datetime.now()
    return now.year - nascido.year - ((now.month, now.day) < (nascido.month, nascido.day))

Há duas, principais, alternativas: 
1. Trabalhar com uma única fila, e dentro desta operar inserções e remanejos conforme necessário para adequar aos três graus de prioridades.
2. Fazer uso de três filas, uma para cada prioridade, mais intuitivo e fácil de implementar.

Neste primeiro momento, acabarei optando pela segunda alternativa.

In [55]:
for row in df.iterrows():
    print('\n', row[1])


 Nome                              Ana Laura
Nascimento                       2000-10-04
Prioritário                           False
Entrada          2024-10-27 19:10:00.098615
Especialidade                  oftalmologia
TMA                                      45
Risco                                 verde
Name: 0, dtype: object

 Nome                                 Oliver
Nascimento                       1968-10-26
Prioritário                           False
Entrada          2024-10-24 22:52:34.098615
Especialidade                    neurologia
TMA                                      45
Risco                                  azul
Name: 1, dtype: object

 Nome                              Valentina
Nascimento                       1961-11-27
Prioritário                            True
Entrada          2024-10-23 20:08:15.098615
Especialidade                   cardiologia
TMA                                      15
Risco                                 verde
Name: 2, dtype: object



In [93]:
fila_normal = Fila()
fila_priori1 = Fila()
fila_priori2 = Fila()

In [96]:
fila_normal.inserir(df.iloc[0])
fila_normal.inserir(df.iloc[1])

In [98]:
fila_normal.exibir()

[Nome                                Eduarda
 Nascimento                       1947-02-23
 Prioritário                            True
 Entrada          2024-10-26 00:08:41.940196
 Especialidade                   cardiologia
 TMA                                      30
 Risco                                 verde
 Name: 0, dtype: object,
 Nome                                Eduarda
 Nascimento                       1947-02-23
 Prioritário                            True
 Entrada          2024-10-26 00:08:41.940196
 Especialidade                   cardiologia
 TMA                                      30
 Risco                                 verde
 Name: 0, dtype: object,
 Nome                               Carolina
 Nascimento                       1984-10-29
 Prioritário                           False
 Entrada          2024-10-23 18:55:00.940196
 Especialidade                  dermatologia
 TMA                                      60
 Risco                               laranja
 Name

In [83]:
df.iloc[0]

Nome                                Eduarda
Nascimento                       1947-02-23
Prioritário                            True
Entrada          2024-10-26 00:08:41.940196
Especialidade                   cardiologia
TMA                                      30
Risco                                 verde
Name: 0, dtype: object

In [ ]:
for row in df.iterrows():
    calc_user_age(df.iloc[21]['Nascimento'])
    print('\n', row[1])

#### A se melhorar
* Evitar dados duplicados em instâncias de objeto Fila()

fontes:
    https://www.camara.leg.br/noticias/630767-PESSOAS-MAIS-IDOSAS-PODERAO-TER-PRIORIDADE-NO-ATENDIMENTO